## **Intructions**

  Il faut creer un repertoire "packages" puis y placer les fichiers librairies "*.py"

Import des librairies

In [2]:
from packages.data_processing import *
from packages.graphics import *
import pandas as pd

# **1. Librairie data_processing**

  Fonctions disponibles :
  - copy_data_from_drive
  - load_train
  - label_parsing
  - one_hot_encoding
  - get_labels

## **Exemples d'utilisation**

### *copy_data_from_drive*

Copie des donnees depuis le drive.<br>
Renvoie VRAI si l'operation a reussie, FAUX si un probleme (repertoire ou fichier inexistant) a ete rencontre<br>
Les arguments "from_path", "filename" et "to_path" sont disponibles.

In [3]:
if not copy_data_from_drive():
  print("Erreur au chargement des donnees depuis le drive")

Erreur au chargement des donnees depuis le drive


Chargement du fichier train.csv :<br>
Il est possible de preciser un argument filename, par defaut celui-ci vaut "train.csv"

### *load_train*

In [5]:
train = load_train()
train.head(5)

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,264918 937 266318 937 267718 937 269118 937 27...
1,0011165.jpg_Flower,1355565 1002 1356965 1002 1358365 1002 1359765...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,233813 878 235213 878 236613 878 238010 881 23...


Parsing image / label<br>
L'instruction dropna a volontairement ete traitee hors fonction car il n'est pas certain que nous voulions supprimer les NA dans la phase de segmentation. La laisser dans la fonction restreint par consequent le champs d'action de la fonction et nous obligera a en creer une nouvelle si besoin.<br>
Il est possible de preciser un argument "sep" stipulant sur quel caractere le split doit etre fait, par defaut celui-ci est a "_"


### *label_parsing*

In [6]:
train = label_parsing(train)
train = train.dropna()
train.head(5)

,EncodedPixels,image,label
0,264918 937 266318 937 267718 937 269118 937 27...,0011165.jpg,Fish
1,1355565 1002 1356965 1002 1358365 1002 1359765...,0011165.jpg,Flower
4,233813 878 235213 878 236613 878 238010 881 23...,002be4f.jpg,Fish
5,1339279 519 1340679 519 1342079 519 1343479 51...,002be4f.jpg,Flower
7,67495 350 68895 350 70295 350 71695 350 73095 ...,002be4f.jpg,Sugar


### *one_hot_encoding*

Codage disjonctif **avec** reinitialisation des indexes

In [7]:
train_encoded = one_hot_encoding(train)
train_encoded.head()

,image,Fish,Flower,Gravel,Sugar
0,0011165.jpg,1,1,0,0
1,002be4f.jpg,1,1,0,1
2,0031ae9.jpg,1,1,0,1
3,0035239.jpg,0,1,1,0
4,003994e.jpg,1,0,1,1


Codage disjonctif **sans** reinitialisation des indexes

In [8]:
train_encoded = one_hot_encoding(train, reset_index=False)
train_encoded.head()

,Fish,Flower,Gravel,Sugar
image,,,,
0011165.jpg,1,1,0,0
002be4f.jpg,1,1,0,1
0031ae9.jpg,1,1,0,1
0035239.jpg,0,1,1,0
003994e.jpg,1,0,1,1


### *get_labels*

Generation d'un label complet correspondant aux valeurs superieures ou egales aux seuils indiques

In [4]:
get_labels([0, 0, 1, 1])

'Gravel,Sugar'

In [5]:
get_labels([0, 1, 1, 0], labels=['classe 1', 'Classe 2', 'Classe 3', 'Classe 4'])

'Classe 2,Classe 3'

In [6]:
get_labels([0.6, 0.9, 0.3, 0.9], threshold=[0.72, 0.87, 0.21, 0.83])

'Flower,Gravel,Sugar'

# **2. Librairie graphics**

  Fonctions disponibles :
  - rleToMask
  - surfaceFromRle